In [1]:
import tensorflow as tf 
import numpy as np 
import collections as deque 
from skimage import color 
from skimage import color, transform, exposure
import pickle 
import warnings
import gym 
warnings.filterwarnings('ignore')

In [2]:
def preprocess_observation(self, observation): 
    x_t1 = color.rgb2gray(observation)
    x_t1 = skimage.transform.resize(x_t1, (84,84))
    x_t1 = skimage.exposure.rescale_intensity(x_t1, out_range=(0,255))
    x_t1 = np.uint8(x_t1)
        #x_t1 = x_t1.reshape(1,1, x_t1.shape[0], x_t1.shape[1])
    return x_t1 

In [9]:
class DQNAgent(): 
    def __init__(self, resized_height, resized_width, action_size, session, batch_size):
        tf.global_step = tf.Variable(0, name='Frame', trainable=False) 
        self.frame_inc_op = self.global_step.assign_add(1, use_locking=True)
        self.session = session


        with tf.variable_scope('network'):
            self.action_size = action_size 
            self.state = tf.placeholder(dtype=float32, shape=[None, 84, 84, 1], name='state')
            self.action = tf.placeholder(dtype=int32, shape=[None], name='action')
            self.y = tf.placeholder(dtype=int32, shape=[None], name='y')
            self.reward = tf.placeholder(dtype=float32, shape=[None], name='reward')
        
            self.gamma = 0.9
            self.epsilon = 1.0 
            self.e_decay = 0.99 
            self.e_min = 0.01 
            self.learning_rate = 0.01 
            self.total_reward = 0.0
            self.total_loss = 0.0 
    
            conv1 = tf.contrib.layers.conv2d(self.state, 32, 8, 8, activation_fn=tf.nn.relu)
            conv2 = tf.contrib.layers.conv2d(conv1, 64, 4, 4, activation_fn=tf.nn.relu)
            conv3 = tf.contrib.layers.conv2d(conv2, 64, 3, 3, activation_fn=tf.nn.relu)
            flattened = tf.contrib.layers.flatten(conv3)
            fc1 = tf.contrib.layers.fully_connected(flattened, 512, activation_fn=tf.nn.relu)
            self.prediction = tf.contrib.layers.fully_connected(fc1, self.action_size)
        with tf.variable_scope('optimizer'):
            action_ind = tf.range(batch_sizes) * tf.shape(self.prediction)[1] + self.action
            self.Q = tf.gather(tf.reshape(self.prediction, [-1]), action_ind)
            self.optimizer = tf.train.AdamOptimizer(learning_rate=1e-4)
            self.loss = tf.reduce_mean(tf.squared_difference(self.y, self.Q))
            self.trainable_weights = tf.get_trainable_weights()
            grads = tf.gradients(self.loss, self.trainable_weights)
            grads, _ = tf.clip_by_global_norm(grads, 40)
            grad_vars = list(zip(grads, self.weights))
            self.train_op = self.optimizer.apply_gradients(grad_vars)
    def frame(self): 
        return self.global_step.eval(session=self.sess)
    def update_target(self): 
        self.sess.run(self.target_update)
    def predict_rewards(self, state): 
        return self.sess.run(self.q_values, {self.state, state}).flatten()
    def predict_target(self, state): 
        return np.max(self.sess.run(self.target_q_values, {self.target_state: state}).flatten())
    def train(self, states, actions, rewards): 
        self.sess.run(self.train_op, feed_dict={self.state: state, self.action: action, self.reward: rewards})
    def frame_increment(self): 
        self.frame_inc_op_eval(session=self.sess)

In [12]:
#Cite 
class AgentSummary:
    """Helper wrapper for summary tensorboard logging"""
    def __init__(self, logdir, agent, env_name):
        """ :param logdir: path to the log directory
            :param agent: agent class-wrapper
            :param env_name: environment name"""
        with tf.variable_scope('summary'):
            self.agent = agent
            self.last_time = time.time()
            self.last_frames = self.agent.frame
            scalar_tags = ['fps', 'episode_avg_reward', 'avg_q_value',
                           'epsilon', 'total_frame_step']
            self.writer = tf.train.SummaryWriter(logdir, self.agent.sess.graph)
            self.summary_vars = {}
            self.summary_ph = {}
            self.summary_ops = {}
            for k in scalar_tags:
                self.summary_vars[k] = tf.Variable(0.)
                self.summary_ph[k] = tf.placeholder('float32', name=k)
                self.summary_ops[k] = tf.scalar_summary("%s/%s" % (env_name, k), self.summary_vars[k])
            self.update_ops = []
            for k in self.summary_vars:
                self.update_ops.append(self.summary_vars[k].assign(self.summary_ph[k]))
            self.summary_op = tf.merge_summary(list(self.summary_ops.values()))

    def write_summary(self, tags):
        tags['fps'] = (self.agent.frame - self.last_frames) / (time.time() - self.last_time)
        self.last_time = time.time()
        self.last_frames = self.agent.frame
        self.agent.sess.run(self.update_ops, {self.summary_ph[k]: v for k, v in tags.items()})
        summary = self.agent.sess.run(self.summary_op, 
                                      {self.summary_vars[k]: v for k, v in tags.items()})
        self.writer.add_summary(summary, global_step=self.agent.frame)